In [3]:
import os
import swat
import getpass
from sasctl import Session
from sasctl.tasks import register_model, publish_model


Connect to SAS platform

In [4]:
password = getpass.getpass ('Enter your password : ')
hostname = "https://viya4-s2.zeus.sashq-d.openstack.sas.com/"
os.environ["CAS_CLIENT_SSL_CA_LIST"] = r"C:\Users\sbxmip\.vscode\trustedcerts.pem"
conn = swat.CAS('https://viya4-s2.zeus.sashq-d.openstack.sas.com/cas-shared-default-http', username='sbxmip',password=password)
Session(hostname, 'sbxmip', password)


Perform Simple Table Operations

In [7]:
EU_ETS_tbl = conn.CASTable('EU_ETS_CURATED', caslib='EBA')
EU_ETS_tbl.columnInfo()
EU_ETS_tbl.head(5)

Fit a linear regression model using the GLM Action set

In [ ]:
conn.loadActionSet('regression')
conn.regression.glm( 
    table = dict(name = EU_ETS_tbl, where = 'year_comp = 2021'), 
    classVars=['Country_id','nace_desc','activity_desc'],
    model={'depVar':'verified',
                   'effects':['Country_id', 'activity_desc', 'nace_desc']
                  },
    store = dict(name='reg_model', replace=True),
    selection = {"method":"BACKWARD"}
)


Register and Publish GLM Model

In [ ]:
astore = conn.CASTable('reg_model')
model = register_model(astore, 'Glm_2021_back', 'Digit Project', force=True)
module = publish_model(model, 'maslocal')


Score first row

In [ ]:
# Pass a row of data to MAS and receive the predicted result.
first_row = EU_ETS_tbl.head(1)
result = module.score(first_row)
print(result)